In [9]:
import openai
from projects.chatgpt.agents.LLMFunctionsAgent import LLMFunctionsAgent

# configure the client's secret key
with open('secret-key.txt', 'r') as file:
    secret_key = file.read().strip()
openai.api_key = secret_key

max_tokens = 4092

# automatically reload changes to imports
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import time
def prompt(question):
    start_time_seconds = time.time()
    print(f"asking chatgpt: {question}")
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", #"text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions accurately, without making up facts."},
            {"role": "user", "content": question}
        ],
    )
    # print(response)
    choices = response.choices
    choice = choices[0]
    message = choice['message']
    answer = message['content']
    print(answer)
    print(f"answer received in {time.time() - start_time_seconds} seconds.")

def stream_prompt(question):
    start_time_seconds = time.time()
    print(f"asking chatgpt: {question}")
    stream = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", #"text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions accurately, without making up facts."},
            {"role": "user", "content": question}
        ],
        stream=True,
    )
    for output in stream:
        choices = output['choices']
        choice = choices[0]
        delta = choice['delta']
        if 'content' in delta:
            print(delta['content'], end='')
    print()
    print(f"answer received in {time.time() - start_time_seconds} seconds.")

In [11]:
stream_prompt("""
Who is user Jason McAffee?
""")

asking chatgpt: 
Who is user Jason McAffee
I'm sorry, but I don't have access to information about specific users unless they provide that information to me.
answer received in 4.245713949203491 seconds.


In [10]:

# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_for_knowledge_retrieval.ipynb
import json 

llm_functions_agent = LLMFunctionsAgent()

def function_call_prompt(question):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_user_details",
                "description": "Retrieves information for a user based on the combination of their first and last names.  It returns information about the user's age, location, and profession.",
                "parameters":{
                    "type": "object",
                    "properties": {
                        "first_name": {
                            "type": "string",
                            "description": "First name of the user to get details of."
                        },
                        "last_name": {
                            "type": "string",
                            "description": "Last name of the user to get details of."
                        }
                    },
                    "required": ["first_name", "last_name"]
                }
            },

        },
    ]

    start_time_seconds = time.time()
    print(f"asking chatgpt: {question}")
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions accurately, without making up facts."},
            {"role": "user", "content": question}
        ],
        tools=tools
    )
    print(response)
    
    llm_functions_agent.call_appropriate_function_based_on_llm_response(response)

    print(f"answer received in {time.time() - start_time_seconds} seconds.")


init


In [17]:
%autoreload

function_call_prompt("""
Who is user Jason McAffee?
""")

asking chatgpt: 
Who is user Jason McAffee?
{
  "id": "chatcmpl-8PIaDIA8KgwbRVE9BZdRCxZTolDd3",
  "object": "chat.completion",
  "created": 1701040325,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "tool_calls": [
          {
            "id": "call_rlOzREqlNtC6EjPLBzdY6Kb7",
            "type": "function",
            "function": {
              "name": "get_user_details",
              "arguments": "{\n  \"first_name\": \"Jason\",\n  \"last_name\": \"McAffee\"\n}"
            }
          }
        ]
      },
      "finish_reason": "tool_calls"
    }
  ],
  "usage": {
    "prompt_tokens": 123,
    "completion_tokens": 27,
    "total_tokens": 150
  }
}
get_function_calls_details_from_llm_response called for response:
{
  "id": "chatcmpl-8PIaDIA8KgwbRVE9BZdRCxZTolDd3",
  "object": "chat.completion",
  "created": 1701040325,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
    